In [4]:
import gymnasium as gym
import numpy as np
import loco_mujoco
from loco_mujoco import LocoEnv


env = gym.make("LocoMujoco", env_name="UnitreeA1.simple")
mdp = LocoEnv.make("UnitreeA1.simple.perfect")

# env = gym.make('HalfCheetah-v4')

In [4]:
import jax
import jax.numpy as jnp

x = jnp.arange(5)
print(x)
jnp.save('test_save.npy', x)

[0 1 2 3 4]


In [30]:
import flax
# Load the TrainState from a file
def load_train_state(save_path, state_structure):
    with open(save_path, 'rb') as f:
        state_dict = flax.serialization.from_bytes(state_structure, f.read())
    return state_dict

In [2]:
from flax.training import checkpoints
import math
import uuid
from copy import deepcopy
from dataclasses import asdict, dataclass
from functools import partial
from typing import Any, Callable, Dict, Sequence, Tuple, Union

import chex
#import d4rl  # noqa
import flax.linen as nn
import gymnasium as gym
# import metagym.quadrupedal
import jax
import jax.numpy as jnp
import numpy as np
import optax
import pyrallis
import wandb
import flax
from flax.core import FrozenDict
from flax.training.train_state import TrainState
from flax.training import checkpoints
from tqdm.auto import trange

from rebrac_Fetch import pytorch_init, identity, DetActor, ActorTrainState, ReplayBuffer

actor_learning_rate: float = 1e-3
hidden_dim: int = 256
actor_n_hiddens: int = 3
critic_n_hiddens: int = 3
actor_ln: bool = False

dataset_name = '/home/nikisim/Mag_diplom/CORL/data/FetchReachDense.npy'

buffer = ReplayBuffer()
buffer.create_from_d4rl(
    dataset_name, False, False
)

init_state = buffer.data["states"][0][None, ...]
init_action = buffer.data["actions"][0][None, ...]

key = jax.random.PRNGKey(seed=51)
key, actor_key, critic_key = jax.random.split(key, 3)

# Load the TrainState from a file
def load_train_state(save_path, state_structure):
    with open(save_path, 'rb') as f:
        state_dict = flax.serialization.from_bytes(state_structure, f.read())
    return state_dict

def create_train_state(actor_module, actor_key, init_state, actor_learning_rate):
    return ActorTrainState.create(
        apply_fn=actor_module.apply,
        params=actor_module.init(actor_key, init_state),
        target_params=actor_module.init(actor_key, init_state),
        tx=optax.adam(learning_rate=actor_learning_rate),
    )

save_path = '/home/nikisim/Mag_diplom/CORL/data/saved_models/FetchReach/actor_state90.pkl'

actor_module = DetActor(
        action_dim=init_action.shape[-1],
        hidden_dim=hidden_dim,
        layernorm=actor_ln,
        n_hiddens=actor_n_hiddens,
    )

dummy_state = create_train_state(actor_module, actor_key, init_state, actor_learning_rate)

actor = load_train_state(save_path, dummy_state)


-----------Dataset loaded---------------


In [4]:

print(actor.target_params)

{'params': {'Dense_0': {'kernel': array([[-0.14602408, -0.22698468, -0.08548546, ..., -0.04975212,
        -0.12742001, -0.2578424 ],
       [-0.33556554,  0.14237517,  0.03783202, ...,  0.08671832,
         0.01024811,  0.1053289 ],
       [ 0.01355439, -0.01968372, -0.07256222, ...,  0.10857463,
         0.10180728,  0.19010435],
       ...,
       [-0.07764928, -0.24640918, -0.14659715, ..., -0.22207695,
        -0.06397453, -0.1294698 ],
       [ 0.06875625, -0.00241721, -0.0972665 , ...,  0.20183265,
         0.22510295, -0.01366103],
       [ 0.02791828, -0.09984249,  0.11291629, ...,  0.21878201,
         0.08490983, -0.24760768]], dtype=float32), 'bias': array([ 0.09134793,  0.1       ,  0.1       ,  0.1       ,  0.00185449,
        0.00843881,  0.12572853,  0.1       ,  0.1       ,  0.1       ,
        0.01489196,  0.1       ,  0.1       , -0.08207791,  0.09432369,
        0.02702796,  0.1       ,  0.03410022,  0.1       ,  0.09503895,
        0.10266239,  0.12366823,  0.02314

In [6]:
import gymnasium as gym
import numpy as np
from rebrac_Fetch import make_env, wrap_env, evaluate# ReplayBuffer, create_from
# from rebrac_Fetch import actor_action_fn

seed = 41

@jax.jit
def actor_action_fn(params: jax.Array, obs: jax.Array):
    return actor.apply_fn(params, obs)


env = gym.make(f'FetchReachDense-v2', render_mode='human')
env.action_space.seed(seed)
env.observation_space.seed(seed)
env = wrap_env(env, buffer.mean, buffer.std)



for eval_epoch in range(1):
    eval_returns, eval_success = evaluate(
        env,
        actor.params,
        actor_action_fn,
        10,
        seed=41,
    )
env.close()


#eval_env = make_env(config.dataset_name, seed=config.eval_seed)

------------------------------
10 Suceess Episodes out of 10
------------------------------


In [3]:
from rebrac_Fetch import Config
import json
with open('/home/nikisim/Mag_diplom/CORL/data/saved_models/FetchPush/config.json') as json_file:
    config_dict = json.load(json_file)

config = Config.from_dict(Config, config_dict)

In [5]:
config.dataset_name

'/home/nikisim/Mag_diplom/CORL/data/FetchPushDense.npy'

In [17]:


raw_restored = checkpoints.restore_checkpoint(ckpt_dir='/home/nikisim/Mag_diplom/CORL/data/saved_models/flax-checkpointing', target=None)
raw_restored

2024-04-04 16:25:56.845558: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 16:25:56.845620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 16:25:56.867449: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-04 16:25:57.505191: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/nikisim/.local/lib/python3.10/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


{'opt_state': {'0': {'count': array(27500, dtype=int32),
   'mu': {'params': {'Dense_0': {'bias': array([-1.2987372e-02,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
              2.0203024e-02, -1.9912191e-03,  5.8952766e-04,  0.0000000e+00,
              0.0000000e+00,  0.0000000e+00, -3.0621034e-03,  0.0000000e+00,
              0.0000000e+00,  5.6051939e-45,  5.6051939e-45,  3.3811876e-03,
              0.0000000e+00,  5.6051939e-45,  0.0000000e+00,  5.6051939e-45,
             -1.1433272e-02, -8.9011341e-03, -3.5869079e-03,  5.6051939e-45,
              5.6051939e-45, -5.9464830e-04,  5.6051939e-45,  5.6051939e-45,
             -5.7158493e-03,  5.6051939e-45,  5.6051939e-45,  0.0000000e+00,
              0.0000000e+00,  5.6051939e-45,  5.6051939e-45,  5.6051939e-45,
             -2.7207106e-03,  0.0000000e+00,  3.9559626e-04,  9.3925875e-03,
              5.6051939e-45,  5.6051939e-45, -1.0684513e-02,  5.6051939e-45,
              8.5089868e-04,  5.6051939e-45,  5.6051939e-45, 

In [5]:
def reformat_dataset(dataset):

    dataset['observations'] = dataset.pop('states')
    dataset['actions'] = dataset.pop('actions')
    dataset['next_observations'] = dataset.pop('next_states')
    dataset['rewards'] = dataset.pop('rewards')
    dataset['terminals'] = dataset.pop('last')

    del dataset['absorbing']

    return dataset

In [2]:
import numpy as np

dataset = np.load('dataset_unitree_ground2.npy', allow_pickle=True).item()

In [3]:
dataset['observations'].shape

(880000, 37)

In [5]:
dataset['rewards'][:1000]

array([2.2544324, 6.3797736, 4.4776363, 4.0452647, 4.582766 , 5.0749784,
       4.974747 , 5.8588576, 6.4720454, 6.1751623, 6.2279344, 5.965443 ,
       6.299403 , 5.9506245, 4.6406603, 4.8965163, 5.722931 , 5.6281996,
       5.562319 , 6.055037 , 6.2942204, 5.226195 , 5.948885 , 6.503476 ,
       6.765685 , 6.0895634, 6.526589 , 6.4900637, 6.6746902, 6.45724  ,
       6.2820606, 5.502824 , 6.1139526, 5.2736726, 4.0653906, 4.3741627,
       5.524103 , 6.0509505, 5.7410216, 6.535557 , 6.6985726, 7.052505 ,
       6.7921224, 6.897555 , 6.4473634, 6.5185194, 6.60545  , 6.7146816,
       6.5233507, 6.37828  , 6.4578023, 6.8582006, 5.836542 , 5.2568336,
       5.771411 , 6.566861 , 6.488855 , 6.2971597, 6.524319 , 6.2879934,
       6.2534723, 6.539673 , 6.8222747, 6.4016824, 6.758068 , 6.6446934,
       6.8033857, 6.6996503, 6.7289133, 5.628211 , 6.3594384, 5.462114 ,
       4.7341995, 4.4948487, 5.62541  , 6.310379 , 6.064765 , 6.017187 ,
       6.660936 , 6.9479465, 6.769294 , 6.6723695, 

In [26]:
state

(array([-1.75222744e-01, -7.73589914e-03, -8.11422792e-03,  9.87607437e-04,
         1.83854901e-01,  9.15128876e-01, -2.26819727e+00, -6.37933476e-02,
         9.34214899e-01, -1.91677393e+00, -4.05785975e-02,  9.36312879e-01,
        -1.87045267e+00,  1.60763861e-01,  9.16133264e-01, -2.15280149e+00,
         2.44099928e-01,  3.62274562e-02,  1.94768716e-02,  2.03384910e-01,
         8.99440417e-02, -7.28740115e-03, -8.17263219e-01, -1.93174492e+00,
         2.00346657e+00, -3.39504673e-01,  9.03097584e-01, -2.70741528e-02,
        -3.43999288e-01,  7.30498063e-01,  2.36305918e-01, -5.87035927e-01,
        -2.58242325e+00,  1.63759636e+00,  1.00000000e+00,  0.00000000e+00,
         2.36557250e-01]),
 {})

In [3]:
import numpy as np
from loco_mujoco import LocoEnv
import gymnasium as gym


# create the environment and task together with the reward function
env = gym.make("LocoMujoco", env_name="UnitreeA1.simple.perfect")
# env = gym.wrappers.RecordVideo(env, f"videos/unitreeA1", episode_trigger = lambda x: x % 10 == 0)
action_dim = env.action_space.shape[0]

env.reset()
env.render()
terminated = False
i = 0

while True:
    if i == 1000 or terminated:
        env.reset()
        i = 0
    action = np.random.randn(action_dim)
    nstate, reward, terminated, truncated, info = env.step(action)

    # HERE is your favorite RL algorithm

    env.render()
    i += 1

/home/nikisim/.local/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


KeyboardInterrupt: 

In [14]:

n_episodes = 5
for _ in range(n_episodes):
    state, done = env.reset(), False
    env.render()
    episode_reward = 0.0
    while not done:
        # action = actor.act(state, device)
        action = env.action_space.sample()
        state, reward, done, _, _ = env.step(action)
        env.render()
        episode_reward += reward
    # episode_rewards.append(episode_reward)
    env.close()
# Each task is associated with a dataset
# dataset contains observations, actions, rewards, terminals, and infos
# dataset = env.get_dataset()
# print(dataset['observations']) # An N x dim_observation Numpy array of observations

# # Alternatively, use d4rl.qlearning_dataset which
# # also adds next_observations.
# dataset = d4rl.qlearning_dataset(env)

/home/nikisim/.local/lib/python3.10/site-packages/gym/envs/mujoco/mujoco_env.py:360: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("HalfCheetah-v4", render_mode="rgb_array")
  gym.logger.warn(
/home/nikisim/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


KeyboardInterrupt: 